In [29]:
import os
import subprocess
from pathlib import Path
import numpy as np
import operator
import json
import math
import resource

In [30]:
DIVISIONS = ['div1', 'div2', 'all']

DIV_THRESHOLD = 1650

In [31]:
division = 'div2'

In [32]:
time_limit = 5
memory_limit = 256 * 1024 * 1024

In [33]:
all_students = ['94101737', '94101975', '95105657', '95106494', '95108008', '95109242', '95109383', '96101262', '96101913', '96104881', '96105553', '96105575', '96105626', '96105637', '96105672', '96105683', '96105712', '96105807', '96105829', '96105853', '96105864', '96105875', '96105886', '96105948', '96105983', '96106033', '96106077', '96106117', '96109588', '96109599', '96109606', '96109663', '96109714', '96109736', '96109747', '96109758', '96109899', '96109906', '96110067', '96110129', '96110797', '97100581', '97102044', '97102441', '97105696', '97105771', '97105844', '97105855', '97106262', '97106295', '97106419', '97110155', '97110206', '97213093', '98209048']

In [34]:
import shutil

working_directory = os.path.abspath('')
os.chdir(working_directory)

submissions_path = Path(os.path.join(working_directory, 'submissions'))

tests_path = Path(os.path.join(working_directory, 'tests'))

executables_path = os.path.join(working_directory, f'executables-{division}')

rating_file = 'rating.json'

if os.path.exists(executables_path):
    shutil.rmtree(executables_path)

os.mkdir(executables_path)
    
outputs_path = Path(os.path.join(working_directory, f'outputs-{division}'))
    
if os.path.exists(outputs_path):
    shutil.rmtree(outputs_path)    
    
os.mkdir(outputs_path)

In [35]:
students = list(os.walk(submissions_path))[0][1]

if division != 'div1':
    students += ['dummy']

try:
    with open(rating_file, 'r', encoding='utf-8') as fp:
        rate = json.load(fp)

    for student in students:
        if student not in rate:
            print(f'FATAL ERROR: {student} has no rating!')

    for std in rate:
        if std not in students:
            print(f'{std} did not submit at this round.')
            students.append(std)
except FileNotFoundError:
    print('No rating file.')

if division == 'div1':
    students = list(filter(lambda x: rate[x] >= DIV_THRESHOLD, students))
elif division == 'div2':
    students = list(filter(lambda x: rate[x] < DIV_THRESHOLD, students))
            
final_results = {student: {'ranking': None, 'tests': {}, 'final_score': 0, 'prev_rating': 0, 'new_rating': 0} for student in students}

96109906 did not submit at this round.
97106295 did not submit at this round.
96109736 did not submit at this round.
97213093 did not submit at this round.


In [36]:
print(division, 'contest')
print(f'{len(students)} students we have: {students}')

div2 contest
32 students we have: ['97102044', '97106419', '96104881', '96105875', '97105771', '96106033', '96105626', '96110797', '96109606', '96110129', '97106262', '96105712', '96105575', '96105948', '96105983', '96105553', '97102441', '94170733', '97105696', '98209048', '96105683', '95109383', '96101262', '96106077', '96105807', '96109588', '97110155', 'dummy', '96109906', '97106295', '96109736', '97213093']


In [37]:
print(submissions_path)
print(working_directory)
print(tests_path)
print(executables_path)

/Users/kianooshabbasi/Dropbox/DA99/Covid Challenge/Round-4/judge/submissions
/Users/kianooshabbasi/Dropbox/DA99/Covid Challenge/Round-4/judge
/Users/kianooshabbasi/Dropbox/DA99/Covid Challenge/Round-4/judge/tests
/Users/kianooshabbasi/Dropbox/DA99/Covid Challenge/Round-4/judge/executables-div2


In [38]:
all_tests = list(map(lambda path: os.path.basename(path).split('in')[1].split('.txt')[0], tests_path.rglob('*.txt')))

In [40]:
print('Compiling codes...')

errors = 0

for student in students:
    cpp_files = list(Path(os.path.join(submissions_path, student)).rglob('*.c*'))
    ans = [] 
    for i in cpp_files:
        if i.name.endswith(".cxx") or i.name.endswith(".cpp"):
            ans.append(i)
    cpp_files = ans
    
    if len(cpp_files) != 1:
        print(f'Something wrong with {student}\'s files. Could not find the cpp file')
        final_results[student]['tests'] = {test: {'score': 0, 'cost': None, 'status': 'No Submission'} for test in all_tests}
        errors += 1
        continue

    cpp_file = cpp_files[0]
    output_file = os.path.join(submissions_path, student, 'output-runnable')
    
    process = subprocess.run(['g++', cpp_file, '-std=c++17', '-O2', '-o', output_file], capture_output=True)
    if process.returncode:
        print(f'Could not compile {student}\'s code.')
        print(f'*******************')
        print(process.stderr.decode()[:300])
        print(f'*******************')              
        final_results[student]['tests'] = {test: {'score': 0, 'cost': None, 'status': 'Compiler Error'} for test in all_tests}
        errors += 1
        continue
    
    
    shutil.move(output_file, os.path.join(executables_path, f'{student}.exec'))

print(f'Compilation finished for {len(students)} codes: {errors} error(s).')
print('\n=================================================\n')

Compiling codes...
Something wrong with dummy's files. Could not find the cpp file
Something wrong with 96109906's files. Could not find the cpp file
Something wrong with 97106295's files. Could not find the cpp file
Something wrong with 96109736's files. Could not find the cpp file
Something wrong with 97213093's files. Could not find the cpp file
Compilation finished for 32 codes: 5 error(s).




In [41]:
print('Executing codes...')

def limit_virtual_memory():
    resource.setrlimit(resource.RLIMIT_AS, (memory_limit, memory_limit))


for executable in list(Path(executables_path).rglob('*.exec')):    
    student_id = os.path.basename(executable).split('.exec')[0]
        
    print(f'Running {student_id}\'s codes')        
        
    for test_file in tests_path.rglob('*.txt'):
        test_name = os.path.basename(test_file).split('.txt')[0]
        
        test_num = test_name[2:]
        
        output_dir = os.path.join(outputs_path, test_name)
        os.makedirs(output_dir, exist_ok=True)
        
        output_path = os.path.join(output_dir, f'output-{student_id}.txt')
        
        input_file = open(test_file)
        output_file = open(output_path, 'w')
        
        try:
            subprocess.run([executable], stdin=input_file, stdout=output_file, timeout=time_limit, preexec_fn=limit_virtual_memory, check=True)
            output_file.close()
        except subprocess.TimeoutExpired:
            print(f'Time limit exceeded on test {test_name}')
            final_results[student_id]['tests'][test_num] = {'score': 0, 'cost': None, 'status': 'Time limit'}
            os.remove(output_path)
        except subprocess.CalledProcessError:
            print(f'Runtime error on test {test_name}')
            final_results[student_id]['tests'][test_num] = {'score': 0, 'cost': None, 'status': 'Runtime Error'}
            os.remove(output_path)
        
        input_file.close()
        
print('\n=================================================\n')

Executing codes...
Running 97105696's codes
Running 97106262's codes
Time limit exceeded on test input19
Time limit exceeded on test input18
Time limit exceeded on test input20
Time limit exceeded on test input11
Time limit exceeded on test input12
Time limit exceeded on test input16
Time limit exceeded on test input17
Time limit exceeded on test input15
Time limit exceeded on test input14
Running 97105771's codes
Running 96105575's codes
Running 96105626's codes
Running 96101262's codes
Running 97110155's codes
Running 96105807's codes
Running 97106419's codes
Runtime error on test input1
Runtime error on test input20
Runtime error on test input9
Runtime error on test input13
Runtime error on test input12
Runtime error on test input15
Runtime error on test input14
Running 98209048's codes
Running 96105683's codes
Running 96105553's codes
Running 96105712's codes
Running 96109588's codes
Running 97102441's codes
Running 96106077's codes
Running 96105948's codes
Running 96105875's codes

In [45]:
checker_files = list(Path(working_directory).rglob('checker.cpp')) 
if len(checker_files) != 1:
    print("Please provide one checker.cpp file")

judge = os.path.join(working_directory, 'judge.exec')    

process = subprocess.run(['g++', checker_files[0], '-std=c++14', '-O2', '-o', judge], capture_output=True)
if process.returncode:
    print(f'Could not compile {student}\'s code:')
    print(process.stderr.decode()[:300])

In [44]:
cost = {}
for s in students:
    cost[s] = {}

for test in list(Path(outputs_path).rglob('in*')):
    this_test = os.path.basename(test) 
    test_num = this_test[2:]
    
    input_file = os.path.join(tests_path, this_test + '.txt')
    
    valid_submissions = list(test.rglob('output*.txt'))
    print("Seeing submissions output for test ", test_num, "\n")

    for student_output in valid_submissions:
        student_id = student_output.name.split('output-')[1].split('.txt')[0]
        
        try:
            out_path = os.path.join(test, f'checker-{student_id}.txt')
            log_path = os.path.join(test, f'log-{student_id}.txt')
            out_file = open(out_path, 'w')
            log_file = open(log_path, 'w')
            
            subprocess.run([judge, input_file, student_output], timeout=2, stdout=out_file, stderr=log_file, check=True)
            
            with open(out_path, "r") as f:
                checker_out = f.read()
            
            out_file.close()
            log_file.close()
            
            cost[student_id][test_num] = int(checker_out)
            print(f"{student_id}'s cost is {checker_out}")
        except subprocess.TimeoutExpired:
            print(f'Time limit exceeded on checker. its not normal')
            os.remove(log_file)
        except subprocess.CalledProcessError:
            print(f'Runtime error on checker. its not normal')
            os.remove(log_file)
        
        

Seeing submissions output for test  put16 

96105683's cost is 547304

96105875's cost is 21460

96110129's cost is 799496

96105807's cost is 799496

96106077's cost is 16206

97105696's cost is 42328

96105626's cost is 794020

96105948's cost is 21460

96105553's cost is 31228

96104881's cost is 21460

96110797's cost is 797794

96109588's cost is 21460

97106419's cost is 608946

97102441's cost is 26344

97102044's cost is 16206

98209048's cost is 21534

96101262's cost is 25012

96105575's cost is 21534

96105983's cost is 799496

96109606's cost is 21534

95109383's cost is -1

94170733's cost is 798238

96106033's cost is 16206

97110155's cost is 1199244

96105712's cost is 799496

97105771's cost is 18870

Seeing submissions output for test  put11 

96105683's cost is 300730

96105875's cost is 14518

96110129's cost is 446520

96105807's cost is 446520

96106077's cost is 10980

97105696's cost is 28304

96105626's cost is 442799

96105948's cost is 14518

96105553's cost 

98209048's cost is 13166

96101262's cost is 13862

96105575's cost is 13166

96105983's cost is 383496

96109606's cost is 13166

95109383's cost is -1

94170733's cost is 380364

96106033's cost is 9918

97110155's cost is 575244

96105712's cost is 383496

97105771's cost is 11542

Seeing submissions output for test  put15 

96105683's cost is 1210851

96105875's cost is 37054

96110129's cost is 1806528

96105807's cost is 1806528

96106077's cost is 27936

97105696's cost is 73720

96105626's cost is 1797119

96105948's cost is 37054

96105553's cost is 54611

96104881's cost is 37054

96110797's cost is 1804588

96109588's cost is 37054

97102441's cost is 45881

97102044's cost is 27936

98209048's cost is 37151

96101262's cost is 37830

96105575's cost is 37151

96105983's cost is 1806528

96109606's cost is 37151

95109383's cost is -1

94170733's cost is 1805655

96106033's cost is 27936

97110155's cost is 2709792

96105712's cost is 1806528

97105771's cost is 32592

Seein

In [46]:
def prob(ri, rj):
    delta = (rj - ri) / 400.
    return 1.0 / (1.0 + 10**(delta))

def return_student_ranking(all_scores, all_tests, test_weights, use_w = False):
    total_score = {student: 0 for student in students}
    for t in all_tests:
        results = []
        for s in students:
            if not t in all_scores[s]:
                continue
            res = all_scores[s][t]
            if res != -1:
                results.append((res, s))
        results.sort()
        first_three = results
        if len(results) > 3:
            first_three = first_three[:3]
        C = []
        for i in range(len(first_three)):
            C.append(first_three[i][0])
        Cm = np.mean(C)
        for s in students:
            if not t in all_scores[s]:
                continue
            res = all_scores[s][t]
            if res == -1:
                final_results[s]['tests'][t] = {'score': 0, 'cost': None, 'status': 'Invalid'}
                continue
            X = test_weights[t] * min(1, (Cm/res)**(1/3))
            final_results[s]['tests'][t] = {'score': round(X, 3), 'cost': res, 'status': 'Success'}
            total_score[s] += X
    sorted_d = sorted(total_score.items(), key=operator.itemgetter(1), reverse=True)
    rank = []
    eps = 1e-6
    for i, (student, score) in enumerate(sorted_d):
        cnt = 1
        for i2, (student2, score2) in enumerate(sorted_d):
            if score2 - eps > score:
                cnt += 1
        pair_rank = (student, cnt)
        rank.append(pair_rank)
        
        final_results[student]['final_score'] = round(score, 3)
        final_results[student]['ranking'] = cnt
    return rank

def get_new_ratings(rate, standings): # current rates, standing of student ids in latest round
    seed = {}
    for i in rate:
        seed[i] = 1
        for j in rate:
            if i == j:
                continue
            seed[i] += prob(rate[j], rate[i])
    new_rates = {}
    for idx, rank in enumerate(standings): # u: user
        lo, hi = 10, 4000
        u = rank[0]
        place = rank[1]
        m_i = math.sqrt((place) * seed[u])
        while(hi - lo > 1):
            mid = (hi + lo)//2
            new_seed = 1
            for j in rate:
                if j == u:
                    continue
                new_seed += prob(rate[j], mid)
            
            if new_seed < m_i: # over valued rating
                hi = mid
            else:
                lo = mid
        R = (hi + lo) * 0.5
        d_i = (R - rate[u]) * 0.5
        new_rates[u] = int(rate[u] + d_i)
    return new_rates


def update_ratings():
    rating_files = list(Path(working_directory).rglob(rating_file))
    rate = {}
    if len(rating_files) == 0:
        print("NO ratings will set every one to 1500")
        for s in students:
            rate[s] = 1500
    else:
        with open(rating_file, 'r', encoding='utf-8') as fp:
            rate = json.load(fp)
        rate = {key: value for key, value in rate.items() if key in final_results}
    print("previous rates -----")
    print(rate)
    for x, y in rate.items():
        final_results[x]['prev_rating'] = y
    standings = return_student_ranking(cost, all_tests, {t: 100 for t in all_tests}, False)
    print(standings)
    new_rates = get_new_ratings(rate, standings)
    destination = f"rating-{division}.json"
    print("new rates -----")
    print(new_rates)
    for x, y in new_rates.items():
        final_results[x]['new_rating'] = y
    with open(destination, 'w', encoding='utf-8') as fp:
        json.dump(new_rates, fp)

In [47]:
update_ratings()

previous rates -----
{'96109588': 1648, '97106419': 1648, '96109906': 1620, '96105626': 1610, '97102044': 1614, '96105948': 1593, '96101262': 1626, '97105771': 1576, '96110797': 1543, '96109606': 1610, '96105712': 1558, '96105983': 1479, '97105696': 1598, '96110129': 1476, '97102441': 1584, '96106077': 1548, '96105875': 1473, '98209048': 1473, '96105807': 1451, '96105683': 1457, '97106262': 1486, '96104881': 1454, '96105575': 1409, '96105553': 1463, '94170733': 1435, '97106295': 1426, '96106033': 1449, '96109736': 1463, '95109383': 1401, '97110155': 1413, '97213093': 1380, 'dummy': 1376}
[('97102044', 1), ('96106033', 1), ('96106077', 1), ('97105771', 4), ('96104881', 5), ('96105875', 5), ('96105948', 5), ('96109588', 5), ('96109606', 9), ('96105575', 9), ('98209048', 9), ('96101262', 12), ('97102441', 13), ('96105553', 14), ('97105696', 15), ('96105683', 16), ('96105626', 17), ('96110797', 18), ('94170733', 19), ('96110129', 20), ('96105712', 20), ('96105983', 20), ('96105807', 20), (

In [48]:
data = sorted(final_results.items(), key= lambda item: item[1]['ranking'])

In [49]:
import xlsxwriter


def export_excel(data):

    workbook = xlsxwriter.Workbook(f'results-{division}.xlsx')
    worksheet = workbook.add_worksheet()
    cell_format1 = workbook.add_format({'bg_color': 'green'})
    cell_format2 = workbook.add_format({'bg_color': 'red'})

    col = 0
    worksheet.write(0, 0, "Ranking")
    worksheet.write(0, 1, "Student ID")
    worksheet.write(0, 2, "Final score")
    worksheet.write(0, 3, "Prev rating")
    worksheet.write(0, 4, "Rating change")
    worksheet.write(0, 5, "New rating")
    worksheet.write(0, 6, "")

    for i in range(1, len(all_tests) + 1):
        worksheet.write(0, i + 6, f"Test {i} score\n(cost)")
        
    for i in range(len(data)):
        col = 0
        worksheet.write(2*i+1, col, data[i][1]["ranking"])
        worksheet.write(2*i+2, col, "")
        col += 1
        worksheet.write(2*i+1, col, data[i][0])
        worksheet.write(2*i+2, col, "")
        col += 1
        worksheet.write(2*i+1, col, data[i][1]["final_score"])
        worksheet.write(2*i+2, col, "")
        col += 1
        worksheet.write(2*i+1, col, data[i][1]["prev_rating"])
        worksheet.write(2*i+2, col, "")
        col += 1
        worksheet.write(2*i+1, col, data[i][1]["new_rating"] - data[i][1]["prev_rating"])
        worksheet.write(2*i+2, col, "")
        col += 1
        worksheet.write(2*i+1, col, data[i][1]["new_rating"])
        worksheet.write(2*i+2, col, "")
        col += 1
        worksheet.write(2*i+1, col, "")
        worksheet.write(2*i+2, col, "")
        col += 1
        for num in sorted(map(lambda x: x[3:], data[i][1]["tests"]), key=int):
            worksheet.write(2 * i + 1, col, data[i][1]["tests"][f"put{num}"]["score"])
            if data[i][1]["tests"][f"put{num}"]['status'] == "Success":
                worksheet.write(2 * i + 2, col, f'({data[i][1]["tests"][f"put{num}"]["cost"]})')
            else:
                worksheet.write(2 * i + 2, col, f'({data[i][1]["tests"][f"put{num}"]["status"]})')
        
            col += 1

    workbook.close()

In [50]:
export_excel(data)

In [51]:
os.remove(judge)